In [1]:
import warnings 
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# https://www.kaggle.com/datasets/putdejudomthai/ecommerce-ab-testing-2022-dataset1

### На сайте Kaggle доступно 2 файла:
    - ab_data.csv, основные данные, 294480 строк
    - countries.csv второстепенные данные с местоположением пользователей, 290586 строк
##### Разница 3894 строк

In [3]:
!ls

ab_data.csv.zip  countries.csv.zip  README.md  Untitled.ipynb


In [4]:
!unzip ab_data.csv.zip
!unzip countries.csv.zip

Archive:  ab_data.csv.zip
  inflating: ab_data.csv             
Archive:  countries.csv.zip
  inflating: countries.csv           


In [5]:
!ls

ab_data.csv	 countries.csv	    README.md
ab_data.csv.zip  countries.csv.zip  Untitled.ipynb


In [6]:
ab_data = pd.read_csv('ab_data.csv')
countries = pd.read_csv('countries.csv')

In [7]:
ab_data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [8]:
countries.head()

,user_id,country
0,834778,UK
1,928468,US
2,822059,UK
3,711597,UK
4,710616,UK


In [9]:
ab_data.shape, countries.shape

((294480, 5), (290586, 2))

In [10]:
df = ab_data.merge(right=countries, how='left', on='user_id')

## Объединив таблицы, я ожидал увидель Null в столбце 'country', но их не оказалось. Тогда я решил посмотреть на количество пользователей, у которых количество просмотров больше одного. Их количество оказалось равным разнице количества строк из таблицы ab_data и countries.

In [11]:
df.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
country         0
dtype: int64

In [12]:
ab_data['user_id'].unique().size, df['user_id'].unique().size

(290585, 290585)

In [13]:
df.groupby('user_id').count()['timestamp'] \
    .sort_values(ascending=False)[:10]

user_id
759899    4
771082    2
679326    2
875210    2
870418    2
714496    2
931907    2
825266    2
706601    2
729669    2
Name: timestamp, dtype: int64

In [14]:
df.groupby('user_id').count()['timestamp'].value_counts()

1    286690
2      3894
4         1
Name: timestamp, dtype: int64

In [15]:
df.head()

,user_id,timestamp,group,landing_page,converted,country
0,851104,11:48.6,control,old_page,0,US
1,804228,01:45.2,control,old_page,0,US
2,661590,55:06.2,treatment,new_page,0,US
3,853541,28:03.1,treatment,new_page,0,US
4,864975,52:26.2,control,old_page,1,US


In [16]:
df[df['group'] == 'control']['landing_page'].value_counts(), \
df[df['group'] == 'treatment']['landing_page'].value_counts()

(old_page    145274
 new_page      1928
 Name: landing_page, dtype: int64,
 new_page    145315
 old_page      1965
 Name: landing_page, dtype: int64)

In [17]:
1928 + 1965 

3893

In [18]:
count_views = df.groupby('user_id').count()['timestamp']
user_id_more_1_views = count_views[count_views > 1].index

In [19]:
user_id_more_1_views.size

3895

In [20]:
df[(df['group'] == 'control') & (~df['user_id'].isin(user_id_more_1_views))]['landing_page'].value_counts()

old_page    143293
Name: landing_page, dtype: int64

In [21]:
df[(df['group'] == 'treatment') & (~df['user_id'].isin(user_id_more_1_views))]['landing_page'].value_counts()

new_page    143397
Name: landing_page, dtype: int64

In [22]:
df[(df['group'] == 'control') & (df['user_id'].isin(user_id_more_1_views))]['landing_page'].value_counts()

old_page    1981
new_page    1928
Name: landing_page, dtype: int64

In [23]:
df[(df['group'] == 'treatment') & (df['user_id'].isin(user_id_more_1_views))]['landing_page'].value_counts()

old_page    1965
new_page    1918
Name: landing_page, dtype: int64

In [24]:
1965 + 1918, 1981 + 1928, 3883 + 3909

(3883, 3909, 7792)

In [25]:
count_views[count_views > 1].sum()

7792

## Таких пользователей нужно удалить из выборки потому, что они могут повлиять на p-value, увеличивая его.

In [26]:
# df with user_id view == 1
df = df[~df['user_id'].isin(user_id_more_1_views)]

In [27]:
df.shape

(286690, 6)

In [28]:
df[(df['group'] == 'control')]['landing_page'].value_counts(), \
df[(df['group'] == 'treatment')]['landing_page'].value_counts()

(old_page    143293
 Name: landing_page, dtype: int64,
 new_page    143397
 Name: landing_page, dtype: int64)

In [29]:
df.groupby('user_id').count()['timestamp'] \
    .sort_values(ascending=False)[:10]

user_id
630000    1
840746    1
840752    1
840751    1
840750    1
840749    1
840748    1
840747    1
840745    1
840733    1
Name: timestamp, dtype: int64

In [30]:
df.head()

,user_id,timestamp,group,landing_page,converted,country
0,851104,11:48.6,control,old_page,0,US
1,804228,01:45.2,control,old_page,0,US
2,661590,55:06.2,treatment,new_page,0,US
3,853541,28:03.1,treatment,new_page,0,US
4,864975,52:26.2,control,old_page,1,US


In [31]:
df.groupby('landing_page').mean()['converted']

landing_page
new_page    0.118726
old_page    0.120173
Name: converted, dtype: float64

In [32]:
df.groupby('landing_page').count()['converted']

landing_page
new_page    143397
old_page    143293
Name: converted, dtype: int64

## Подсчет p-value с помощью proportions_ztest

In [33]:
from statsmodels.stats.proportion import proportions_ztest

In [42]:
a = df[df['landing_page'] == 'old_page']['converted']
b = df[df['landing_page'] == 'new_page']['converted']
zstat, pval = proportions_ztest([a.sum(), b.sum()], [a.size, b.size], alternative='smaller')
zstat, pval

(1.1944850812796501, 0.8838558634708319)

## Подсчет zstat и p-value вручную

In [43]:
m1 = a.mean()
m2 = b.mean()
m = np.concatenate((a.values, b.values)).mean()
n1 = a.size
n2 = b.size

In [44]:
m1, m2, m, n1, n2

(0.12017335110577627, 0.11872633318688676, 0.11944957968537445, 143293, 143397)

In [45]:
zstat = (m1-m2) / np.sqrt( m*(1-m) * (1/n1 + 1/n2) )
zstat

1.1944850812796501

In [46]:
import scipy.stats as sts

In [47]:
pval = sts.norm(0, 1).cdf(zstat)
pval

0.8838558634708319